In [29]:

# Array manipulation
import numpy as np

# Random variables generator
import random as rand

# data processing / .csv manipulation and reading
import pandas as pd


# Data visualization
import seaborn as sns
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import style

# Algorithms / AI
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules


In [30]:
rules = pd.read_csv('rules.csv') # Objects are loaded with infos from .csv

In [31]:
rules.drop('ID', axis='columns', inplace=True)

In [32]:
for column in ['antecedents', 'consequents']:
  for index, row in enumerate( rules[column] ):
    
    rules.loc[index, column] = row.replace('frozenset({', '').replace('})', '').replace(' ', '')

rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,'Mgrd212','Mgrd181',0.760503,0.388510,0.364452,0.479226,1.233497,0.068990,1.174194,0.790392
1,'Mgrd212',"'Mgrd181','Mgrd19'",0.760503,0.317056,0.298923,0.393059,1.239718,0.057801,1.125225,0.807377
2,'Mgrd212',"'Mgrd181','Mgrd223'",0.760503,0.270557,0.270557,0.355760,1.314920,0.064798,1.132254,1.000000
3,'Mgrd06','Mgrd08',0.351526,0.321724,0.265530,0.755363,2.347863,0.152435,2.772579,0.885279
4,'Mgrd08','Mgrd06',0.321724,0.351526,0.265530,0.825335,2.347863,0.152435,3.712669,0.846382
...,...,...,...,...,...,...,...,...,...,...
121,'Mgrd035',"'Mgrd044','Mgrd01'",0.426391,0.303950,0.279354,0.655158,2.155481,0.149752,2.018461,0.934551
122,'Mgrd044',"'Mgrd035','Mgrd01'",0.303950,0.426391,0.279354,0.919079,2.155481,0.149752,7.088463,0.770155
123,"'Mgrd01','Mgrd212'",'Mgrd044',0.444165,0.303950,0.263555,0.593371,1.952201,0.128551,1.711758,0.877523
124,'Mgrd044',"'Mgrd01','Mgrd212'",0.303950,0.444165,0.263555,0.867100,1.952201,0.128551,4.182348,0.700751


In [33]:
rules.loc[101, ['antecedents', 'consequents']]

antecedents    'Mgrd031','Mgrd01','Mgrd212'
consequents                       'Mgrd035'
Name: 101, dtype: object

In [34]:
print(f'''
This:\t\t{rules.values[100][0]+' and '+rules.values[100][1]}
Becomes this: \t{sorted((rules.values[100][0]+','+rules.values[100][1]).split(','))}

This:\t\t{rules.values[101][0]+' and '+rules.values[101][1]}
Becomes this: \t{sorted((rules.values[101][0]+','+rules.values[101][1]).split(','))}

Is equal: {sorted((rules.values[101][0]+','+rules.values[101][1]).split(',')) == sorted((rules.values[100][0]+','+rules.values[100][1]).split(','))}.''')


This:		'Mgrd035' and 'Mgrd031','Mgrd212'
Becomes this: 	["'Mgrd031'", "'Mgrd035'", "'Mgrd212'"]

This:		'Mgrd031','Mgrd01','Mgrd212' and 'Mgrd035'
Becomes this: 	["'Mgrd01'", "'Mgrd031'", "'Mgrd035'", "'Mgrd212'"]

Is equal: False.


In [35]:
rules.loc[:, 'zhangs_metric'].idxmax()

21

In [36]:
# Now, the only thing that matters to understanding the association rules
# above is a list that shortens the association rules that are repeated.

def pruneDataFrame( df ):
  
  organized_rules = []
  
  for index, row in enumerate( df.values ):
    
    organized_rules.append( sorted((row[0]+','+row[1]).split(',')) )
  
  duplicates = []
  
  for rule_index, rule in enumerate(organized_rules):
    duplicates.append(list())
    for comp_index, comparator in enumerate(organized_rules):
      if comparator == rule:
        duplicates[rule_index].append(comp_index)
  
  max_set = set()
  
  for dups in duplicates:
    if len(dups) > 1:
      
      max_index = df.loc[ dups, 'zhangs_metric'].idxmax()

    else:
      max_index = dups[0]
    
    max_set.add(max_index)
  
  return df.loc[ list(max_set) ]

pruned_rules = pruneDataFrame( rules )

In [37]:
pruned_rules.shape

(47, 10)

In [38]:
filename = f'pruned-rules.xlsx'
with open(filename, 'w') as output_handle: # Save as .csv for visualization
  pruned_rules.to_excel(filename, columns=pruned_rules.columns)

c:\Users\tobie\.conda\envs\cope_AI\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\tobie\.conda\envs\cope_AI\Lib\site-packages\openpyxl\writer\excel.py:292: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  workbook.properties.modified = datetime.datetime.utcnow()


In [39]:
indexes = pd.read_csv('indexes.csv')

In [40]:
indexes

,param1,param2,param3,code
0,O município foi atingido pela seca nos últimos...,NaN,NaN,MGRD01
1,O município foi atingido pela seca nos últimos...,Ano em que aconteceu a seca de maior impacto p...,NaN,MGRD02
2,O município foi atingido pela seca nos últimos...,No ano de seca de maior impacto para o municíp...,Perdas financeiras,MGRD031
3,O município foi atingido pela seca nos últimos...,No ano de seca de maior impacto para o municíp...,Perdas humanas,MGRD032
4,O município foi atingido pela seca nos últimos...,No ano de seca de maior impacto para o municíp...,Perdas de animais,MGRD033
...,...,...,...,...
155,Atividades de proteção de defesa civil,Campanhas para conscientizar a população sobre...,NaN,MGRD235
156,Atividades de proteção de defesa civil,Capacitação de profissionais de proteção e def...,NaN,MGRD236
157,Atividades de proteção de defesa civil,Formação de NUDECs,NaN,MGRD237
158,Atividades de proteção de defesa civil,Nenhuma das citadas,NaN,MGRD238


In [41]:
for row in pruned_rules['antecedents']:
  print(row)

'Mgrd212'
'Mgrd212'
'Mgrd212'
'Mgrd06'
'Mgrd08'
'Mgrd212'
'Mgrd212'
'Mgrd212'
'Mgrd212'
'Mgrd223'
'Mgrd212'
'Mgrd212'
'Mgrd212'
'Mgrd223'
'Mgrd212'
'Mgrd212'
'Mgrd212'
'Mgrd212'
'Mgrd208'
'Mgrd208'
'Mgrd01'
'Mgrd035'
'Mgrd031'
'Mgrd01'
'Mgrd035'
'Mgrd01'
'Mgrd01'
'Mgrd01'
'Mgrd01'
'Mgrd01'
'Mgrd11'
'Mgrd01'
'Mgrd01'
'Mgrd01'
'Mgrd01'
'Mgrd01'
'Mgrd01'
'Mgrd035'
'Mgrd01'
'Mgrd01'
'Mgrd035'
'Mgrd01'
'Mgrd01'
'Mgrd01'
'Mgrd035'
'Mgrd01'
'Mgrd01'


In [42]:
for code_index, code in enumerate( indexes['code'] ):
  for col_name in ['antecedents', 'consequents']:
    for row_index, row in enumerate( pruned_rules[col_name] ):
      
      code_name = str(indexes.loc[code_index, 'param1'])
      code_name = code_name+' '+str(indexes.loc[code_index, 'param2'])
      code_name = code_name+' '+str(indexes.loc[code_index, 'param3'])
      
      if type(row) == type(0.55):
        print(row, pruned_rules.index[row_index])
      if "'"+code.title()+"'" in row:
        pruned_rules.loc[pruned_rules.index[row_index], col_name] = row.replace(code.title(), code_name)

In [43]:
pruned_rules['consequents'][2]

"'Em relação ao gerenciamento de riscos de desastres decorrentes de enchentes ou inundações graduais, ou enxurradas ou inundações bruscas, o município possui Mapeamentos de áreas de risco de enchentes ou inundações nan','Em relação a Coordenação Municipal de Proteção e Defesa Civil (COMPDEC) ou órgão similar Conhece o Sistema Integrado de Informações sobre Desastres - S2ID nan'"

In [44]:
filename = f'pruned-rules_labeled.xlsx'
with open(filename, 'w') as output_handle: # Save as .csv for visualization
  pruned_rules.to_excel(filename, columns=pruned_rules.columns)

c:\Users\tobie\.conda\envs\cope_AI\Lib\site-packages\openpyxl\packaging\core.py:99: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()
c:\Users\tobie\.conda\envs\cope_AI\Lib\site-packages\openpyxl\writer\excel.py:292: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  workbook.properties.modified = datetime.datetime.utcnow()


In [45]:
pruned_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"'Em relação à gestão de riscos e de desastres,...",'Em relação ao gerenciamento de riscos de desa...,0.760503,0.388510,0.364452,0.479226,1.233497,0.068990,1.174194,0.790392
1,"'Em relação à gestão de riscos e de desastres,...",'Em relação ao gerenciamento de riscos de desa...,0.760503,0.317056,0.298923,0.393059,1.239718,0.057801,1.125225,0.807377
2,"'Em relação à gestão de riscos e de desastres,...",'Em relação ao gerenciamento de riscos de desa...,0.760503,0.270557,0.270557,0.355760,1.314920,0.064798,1.132254,1.000000
3,'O município foi atingido por alagamentos nos ...,'O município foi atingido por enchentes ou inu...,0.351526,0.321724,0.265530,0.755363,2.347863,0.152435,2.772579,0.885279
6,'O município foi atingido por enchentes ou inu...,'No ano de enchente ou inundação gradual de ma...,0.321724,0.272890,0.272890,0.848214,3.108259,0.185095,4.790369,1.000000
7,"'Em relação à gestão de riscos e de desastres,...",'Em relação a Coordenação Municipal de Proteçã...,0.760503,0.472531,0.472531,0.621341,1.314920,0.113170,1.392991,1.000000
8,"'Em relação à gestão de riscos e de desastres,...",'Em relação a Coordenação Municipal de Proteçã...,0.760503,0.357630,0.357630,0.470255,1.314920,0.085651,1.212602,1.000000
9,"'Em relação à gestão de riscos e de desastres,...",'Em relação a Coordenação Municipal de Proteçã...,0.760503,0.292998,0.292998,0.385269,1.314920,0.070172,1.150100,1.000000
10,"'Em relação à gestão de riscos e de desastres,...",'Em relação a Coordenação Municipal de Proteçã...,0.760503,0.368941,0.368941,0.485127,1.314920,0.088360,1.225661,1.000000
11,'Em relação a Coordenação Municipal de Proteçã...,'Em relação a Coordenação Municipal de Proteçã...,0.472531,0.368941,0.297487,0.629559,1.706397,0.123150,1.703536,0.784824
